In [1]:
import random
#import re
from datetime import date
#from urllib.request import urlretrieve
from IPython.display import clear_output
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
#from selenium.webdriver.common.by import By
from lxml import etree
import json
import time
import traceback
import os
from tqdm import tqdm

In [2]:
class crawling:
    
    def __init__(self,):
        self.home_page = "https://batdongsan.so/nha-dat-ban/ha-noi#/"
        self.root = "https://batdongsan.so/nha-dat/"
        self.page = 1
    
    def get_pages(self, page_source):
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        links = [i.getchildren()[0].get("href") for i in page.xpath("""//*[@id="main"]/div/div/section/div/div[1]""")[0].getchildren()]
        links = [link for link in links if link != None]
        return links
    
    def next_page(self):
        self.page += 1
        return self.home_page.removesuffix('#/') + """?page=""" + f"{self.page}" + "#/"
    
    def gather(self, page_source):
        data = {}
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        #get price
        price_path=page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[1]/div[2]/div[2]/strong""")
        if len(price_path)!=0:
            data["price"]=page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[1]/div[2]/div[2]/strong""")[0].text + " " + page.xpath("""//*[@id="main"]/div[1]/div/section[1]/div[1]/div[2]/div[2]/text()[2]""")[0].strip()
        else:
            data["price"]=None
        
        #get address
        if len(bs.find_all("div", {"class": "re-address"}))!=0:
            address = bs.find_all("div", {"class": "re-address"})[0].text.strip()
            data["address"] = address
        else:
            data["address"] = None

        
        #get detailed description
        des_elements = list(bs.find_all("ul", {"class": "re-property"})[0].children)[0:-1:2]
        for i in range(len(des_elements)):
            key_value = des_elements[i].text.strip().split(":")
            key = key_value[0]
            value = key_value[1]
            data[key] = value
            
        #get id
        other_information= list(bs.find_all("ul", {"class": "short-detail-2 list2 clearfix"})[0].children)[0:-1:2]
        try:
            for i in range(len(other_information)):
                key_value = other_information[i].text.strip().split(":")
                key = key_value[0]
                value = key_value[1]
                data[key] = value
            if "Mã tin" in data:
                data["id"] = "batdongsan_so_com_vn_" + str(data["Mã tin"])
            else:
                data["id"] = "batdongsan_so_com_vn_" + str(time.time())
        except Exception as e:
            print(e)
            import pdb
            pdb.set_trace()
        return data
    
    def run(self, start, num_of_pages):
        service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
        driver = webdriver.Edge(service=service)
        url = self.home_page
        self.page = 1
        if start != 1:
            self.page = start
            url = self.home_page.removesuffix('#/') + """?page=""" + f"{self.page}" + "#/"
        if os.path.exists("data/index.json"):
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
            
        with open("log.txt", "w") as f:
            pass
        
        dataset = []
        for i in range(num_of_pages):
            num = 10
            pages = []
            while num > 0:
                try:
                    driver.get(url)
                    time.sleep(0.05)
                    pages = self.get_pages(driver.page_source)
                    driver.delete_all_cookies()
                    break
                except:
                    num -= 1
            if num <= 0:
                url = self.next_page()
                continue
            #driver.close()
            #service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
            #driver = webdriver.Edge(service=service)
            for page in tqdm(pages):
                num = 10
                while num > 0:
                    try:
                        driver.get(page)
                        time.sleep(0.05)
                        data = self.gather(driver.page_source)
                        if data["id"] not in index:
                            dataset.append(data)
                            index.add(data["id"])
                        with open("log.txt", "a") as f:
                            f.write("Success:" + page +"\n")
                        driver.delete_all_cookies()
                        break
                    except Exception as e:
                        print(e)
                        with open("log.txt", "a") as f:
                            f.write(f"Excution{10-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        driver.delete_all_cookies()
                        num -= 1
            url = self.next_page()
        if len(dataset) != 0:
            df = pd.DataFrame(dataset)
            df.to_csv(f"data/{start}_{start+num_of_pages}_{time.time()}.csv", index=False)
        with open("data/index.json", "w") as f:
            json.dump(list(index), f)
        driver.close()
        

In [3]:
craw = crawling()

In [4]:
for i in tqdm(range(203, 3227)):
    craw.run(i, 1)
    clear_output()

  2%|█▎                                                                       | 52/3024 [3:31:08<201:07:46, 243.63s/it]


WebDriverException: Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: MicrosoftEdge=108.0.1462.42)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF7F84E91A2+14306]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF7F8470962+842818]
	(No symbol) [0x00007FF7F8121D90]
	(No symbol) [0x00007FF7F810A143]
	(No symbol) [0x00007FF7F80F9586]
	(No symbol) [0x00007FF7F80FA9AE]
	(No symbol) [0x00007FF7F8191250]
	(No symbol) [0x00007FF7F8185CF0]
	(No symbol) [0x00007FF7F81573B8]
	(No symbol) [0x00007FF7F815638E]
	(No symbol) [0x00007FF7F8157AE4]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF7F83BFDF8+119000]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF7F83AD4D6+42934]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF7F83B06EC+55756]
	(No symbol) [